# Taller 04 Spines Cubicos
## Integrantes:
## - Christian Ayala
## - Jhonn Saeteros


In [8]:
import sympy as sym
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

def cubic_spline_clamped(xs, ys, B0, B1):
    n = len(xs) - 1
    h = [xs[i+1] - xs[i] for i in range(n)]
    
    alpha = [0] * (n+1)
    alpha[0] = 3*(ys[1] - ys[0])/h[0] - 3*B0
    alpha[n] = 3*B1 - 3*(ys[n] - ys[n-1])/h[n-1]
    
    for i in range(1, n):
        alpha[i] = (3/h[i])*(ys[i+1] - ys[i]) - (3/h[i-1])*(ys[i] - ys[i-1])
    
    l = [1] + [0]*n
    mu = [0]*n + [0]
    z = [0]*(n+1)
    
    for i in range(1, n):
        l[i] = 2*(xs[i+1] - xs[i-1]) - h[i-1]*mu[i-1]
        mu[i] = h[i]/l[i]
        z[i] = (alpha[i] - h[i-1]*z[i-1])/l[i]
    
    l[n] = h[n-1]*(2 - mu[n-1])
    z[n] = (alpha[n] - h[n-1]*z[n-1])/l[n]
    
    c = [0]*(n+1)
    b = [0]*n
    d = [0]*n
    a = [ys[i] for i in range(n)]

    c[n] = z[n]
    for j in reversed(range(n)):
        c[j] = z[j] - mu[j]*c[j+1]
        b[j] = ((ys[j+1] - ys[j])/h[j]) - h[j]*(c[j+1] + 2*c[j])/3
        d[j] = (c[j+1] - c[j]) / (3*h[j])
    
    x = sym.Symbol("x")
    splines = []
    for j in range(n):
        S = a[j] + b[j]*(x - xs[j]) + c[j]*(x - xs[j])**2 + d[j]*(x - xs[j])**3
        splines.append(S)
    
    return splines, xs

# Puntos base
xs = [0, 1, 2]
ys = [1, 5, 3]
B0 = 1

# Preparar figura
fig, ax = plt.subplots(figsize=(8, 5))
x_vals = np.linspace(xs[0], xs[-1], 300)
line, = ax.plot([], [], lw=2, label='Spline')
tangent_line, = ax.plot([], [], '--g', label='Tangente final')
points = ax.scatter(xs, ys, color='red')
text_label = ax.text(0.05, 0.9, '', transform=ax.transAxes)

ax.set_xlim(-0.5, 2.5)
ax.set_ylim(-5, 20)
ax.set_title('Spline cúbico variando tangente en x=2')
ax.set_xlabel('x')
ax.set_ylabel('S(x)')
ax.grid()
ax.legend()

# Valores de B1 a recorrer
B1_vals = np.linspace(-10, 10, 100)

def animate(i):
    B1 = B1_vals[i]
    splines, xs_sorted = cubic_spline_clamped(xs, ys, B0, B1)

    y_vals = np.zeros_like(x_vals)
    x_sym = sym.Symbol("x")
    for j in range(len(xs_sorted)-1):
        mask = (x_vals >= xs_sorted[j]) & (x_vals <= xs_sorted[j+1])
        f = sym.lambdify(x_sym, splines[j], 'numpy')
        y_vals[mask] = f(x_vals[mask])

    line.set_data(x_vals, y_vals)

    # Tangente en x = 2
    x0 = xs_sorted[-1]
    f_end = sym.lambdify(x_sym, splines[-1], 'numpy')
    y0 = f_end(x0)
    x_tan = np.array([x0 - 0.5, x0 + 0.5])
    y_tan = y0 + B1 * (x_tan - x0)
    tangent_line.set_data(x_tan, y_tan)

    text_label.set_text(f'B₁ = {B1:.1f}')
    return line, tangent_line, text_label

# Animación
anim = FuncAnimation(fig, animate, frames=len(B1_vals), interval=100, blit=True)

# Guardar como GIF
anim.save('spline_con_tangente_variable.gif', writer=PillowWriter(fps=20))
plt.close()
